In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install abstractions-aimedic

In [3]:
!pip install unrar
!unrar  x "/content/drive/MyDrive/AIMedic_Moazed/Resources/EchoNet-Dynamic_Cases.rar" -d ./

Streaming output truncated to the last 5000 lines.
Extracting  ./EchoNet-Dynamic/Cases/0X93ECC18675FEC51/0X93ECC18675FEC51_ES_gt.raw      89%  OK 
Creating    ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE                 OK
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ED.mhd      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ED.raw      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ED_gt.mhd      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ED_gt.raw      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ES.mhd      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ES.raw      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF2689DBEE/0X93ED1CF2689DBEE_ES_gt.mhd      89%  OK 
Extracting  ./EchoNet-Dynamic/Cases/0X93ED1CF268

In [17]:
!git clone --branch migration_python_gen https://github.com/iamsoroush/lv-seg

Cloning into 'lv-seg'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 181 (delta 70), reused 163 (delta 52), pack-reused 0
Receiving objects: 100% (181/181), 101.35 KiB | 2.30 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [5]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 39.9 MB/s 


In [1]:
import sys
sys.path.append('lv-seg')

In [3]:
from abstractions.utils import load_config_file

config_path='/content/lv-seg/runs/python_gen_mig/config.yaml'
config = load_config_file(config_path)

In [4]:
from src.dataset.dataset import DataLoader

dataset = DataLoader(config,'/content/EchoNet-Dynamic')
test_gen, n_test = dataset.create_test_generator()
train_gen, n_train = dataset.create_training_generator()
val_gen, n_val = dataset.create_validation_generator()

In [5]:
print('n_test : ', n_test)
print('test_gen_len : ', len(next(test_gen)))
print('test_gen_image_shape :', next(test_gen)[0].shape)
print('test_gen_label_shape :', next(test_gen)[1].shape)
print('test_gen_indices :', next(test_gen)[2])
print('n_train : ', n_train)
print('train_gen_len : ', len(next(train_gen)))
print('train_gen_image_shape :', next(train_gen)[0].shape)
print('train_gen_label_shape :', next(train_gen)[1].shape)
print('train_sample_weight :', next(train_gen)[2])
print('n_val : ', n_val)
print('val_gen_len : ', len(next(val_gen)))
print('tval_gen_image_shape :', next(val_gen)[0].shape)
print('val_gen_label_shape :', next(val_gen)[1].shape)
print('val_sample_weight :', next(val_gen)[2])

n_test :  2552
test_gen_len :  3
test_gen_image_shape : (112, 112, 1)
test_gen_label_shape : (112, 112)
test_gen_indices : 21
n_train :  14920
train_gen_len :  3
train_gen_image_shape : (112, 112, 1)
train_gen_label_shape : (112, 112)
train_sample_weight : [1]
n_val :  2576
val_gen_len :  3
tval_gen_image_shape : (112, 112, 1)
val_gen_label_shape : (112, 112)
val_sample_weight : [1]


In [6]:
from src.dataset.augmentor import Augmentor

augmentor = Augmentor(config)
aug_test = augmentor.add_augmentation(test_gen)

In [7]:
print('aug_test_gen_len : ', len(next(aug_test)))
print('aug_test_gen_image_shape :', next(aug_test)[0].shape)
print('aug_test_gen_label_shape :', next(aug_test)[1].shape)
print('aug_test_gen_indices :', next(aug_test)[2])

aug_test_gen_len :  3
aug_test_gen_image_shape : (112, 112, 1)
aug_test_gen_label_shape : (112, 112)
aug_test_gen_indices : 79


In [8]:
from src.dataset.preprocessing import Preprocessor

preprocessor = Preprocessor(config)
train_data_gen, n_iter_train = preprocessor.add_preprocess(train_gen, n_test)
validation_data_gen, n_iter_val = preprocessor.add_preprocess(val_gen, n_val)

In [9]:
print('n_iter_val : ', n_iter_val)
print('validation_data_gen_len : ', len(next(validation_data_gen)))
print('validation_data_gen_image_shape :', next(validation_data_gen)[0].shape)
print('validation_data_gen_label_shape :', next(validation_data_gen)[1].shape)
print('validation_data_gen_indices :', next(validation_data_gen)[2])

n_iter_val :  322
validation_data_gen_len :  3
validation_data_gen_image_shape : (8, 128, 128, 1)
validation_data_gen_label_shape : (8, 128, 128, 1)
validation_data_gen_indices : [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [10]:
from src.model.baseline import UNetBaselineBuilder
from abstractions import Trainer
from abstractions.utils import setup_mlflow
import pathlib
import os


run_name = 'test-1'

repo_root = pathlib.Path('mnist-test')
run_dir = repo_root.joinpath('runs').joinpath(run_name)
config_path = run_dir.joinpath('config.yaml')
#run_dir.mkdir(exist_ok=True)

#sys.path.append(os.path.join('mnist-test', config.src_code_path))

model_builder = UNetBaselineBuilder(config)
MLFLOW_TRACKING_URI = 'mlruns'

trainer = Trainer(config=config, run_dir=pathlib.Path('/content/lv-seg/runs/python_gen_mig'))
mlflow_active_run = setup_mlflow(mlflow_tracking_uri=MLFLOW_TRACKING_URI,
                                 mlflow_experiment_name=config.project_name,
                                 base_dir=run_dir)

In [11]:
trainer.train(model_builder=model_builder,
                active_run=mlflow_active_run,
                train_data_gen=train_data_gen,
                n_iter_train=n_iter_train,
                val_data_gen=validation_data_gen,
                n_iter_val=n_iter_val)

2021/11/27 09:43:09 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/30
319/319 [==============================] - 155s 461ms/step - loss: 0.1932 - acc: 0.9496 - val_loss: 0.3207 - val_acc: 0.9047
Epoch 2/30
319/319 [==============================] - 147s 461ms/step - loss: 0.0973 - acc: 0.9805 - val_loss: 0.0895 - val_acc: 0.9813
Epoch 3/30
319/319 [==============================] - 147s 461ms/step - loss: 0.0762 - acc: 0.9824 - val_loss: 0.0733 - val_acc: 0.9812
Epoch 4/30
319/319 [==============================] - 147s 462ms/step - loss: 0.0643 - acc: 0.9828 - val_loss: 0.0649 - val_acc: 0.9821
Epoch 5/30
319/319 [==============================] - 146s 458ms/step - loss: 0.0556 - acc: 0.9836 - val_loss: 0.0558 - val_acc: 0.9824
Epoch 6/30
319/319 [==============================] - 145s 455ms/step - loss: 0.0546 - acc: 0.9825 - val_loss: 0.0516 - val_acc: 0.9835
Epoch 7/30
319/319 [==============================] - 144s 452ms/step - loss: 0.0467 - acc: 0.9841 - val_loss: 0.0479 - val_acc: 0.9829
Epoch 8/30
319/319 [============================

2021/11/27 10:56:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Changing param values is not allowed. Param with key='batch_size' was already logged with value='8' for run ID='d311a93a583e4c5382d6809baa27eb7f'. Attempted logging new value 'None'.


In [12]:
exported_dir = trainer.export()

In [24]:
import mlflow
from src.evaluation.evaluator import Evaluator
import tensorflow.keras as tfk

exported_model = tfk.models.load_model(trainer.exported_saved_model_path)

mlflow.end_run()
eval_active_run = setup_mlflow(mlflow_tracking_uri=MLFLOW_TRACKING_URI,
                                mlflow_experiment_name=config.project_name,
                                base_dir=run_dir,
                                evaluation=True)

evaluator = Evaluator(config)
eval_report = evaluator.evaluate(data_loader=dataset,
                                preprocessor=preprocessor,
                                exported_model=exported_model,
                                active_run=eval_active_run)
eval_report.describe()

  0%|          | 0/2552 [00:00<?, ?it/s]


AttributeError: ignored